# Предобработка данных

- Удаляем данные *первого года январь-октябрь* и *последнего года ноябрь-декабрь*
- Удаляем лишние признаки: координаты, пути к изображениям, максимальная и минимальные температуры
- Соединяем признаки с таргетами по (`target_year`, `fips`) и (`year`, `target`) соответственно
- Обработка `NaN значений
- Разделяем на `train`, `test`. В качестве `test` возьмем последний год

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

PATH_INTERIM = Path("../../data/interim")
PATH_PROCESSED = Path("../../data/processed")

In [2]:
X = pd.read_csv(PATH_INTERIM / "X.csv")
y = pd.read_csv(PATH_INTERIM / "y.csv")

## Удаляем данные *первого года январь-октябрь* и *последнего года ноябрь-декабрь*

In [3]:
min_year, max_year = X["year"].min(), X["year"].max()
X = X[
    ~((X["year"] == min_year) & (X["month"] < 11))
    | ((X["year"] == max_year) & (X["month"] > 10))
]

## Удаляем лишние признаки

In [4]:
X.drop(
    [
        "lat_lower_left",
        "lon_lower_left",
        "lat_upper_right",
        "lon_upper_right",
        "temperature_max",
        "temperature_avg",
        "images",
    ],
    axis=1,
    inplace=True,
)

In [5]:
display(X.shape)
X.head(5)

(22156, 18)

,year,month,day,fips,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,target_year
3860,2017,11,1,17001,273.936458,0.187208,71.783333,7.191125,3.372792,-2.079917,2.556042,444.233333,0.222625,4.736086e-07,0.310632,0.317329,0.258307,2018
3861,2017,11,1,17003,278.042667,18.158111,90.355556,4.224000,2.226778,-1.714222,0.091889,61.733333,0.099333,3.250581e-04,0.383857,0.324059,0.241359,2018
3862,2017,11,1,17005,275.822167,2.791333,88.300000,5.595833,3.043833,-1.965500,1.833667,111.766667,0.095833,1.284828e-04,0.451369,0.439113,0.399522,2018
3863,2017,11,1,17007,273.233000,6.926667,78.400000,5.291667,2.995000,-1.246667,2.581333,358.500000,0.154000,3.930181e-07,0.339598,0.347218,0.311294,2018
3864,2017,11,1,17009,274.035000,0.126250,73.925000,6.281875,3.229250,-2.013000,2.428250,375.025000,0.204250,8.822729e-07,0.299224,0.304276,0.238118,2018


## Соединяем признаки с таргетами

In [6]:
X["target_year"] = X["target_year"].astype(y["year"].dtype)
data = pd.merge(
    X.drop("year", axis=1),
    y,
    how="inner",
    left_on=["target_year", "fips"],
    right_on=["year", "fips"],
)
display(data.shape)
data.head()

(21020, 19)

,month,day,fips,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,target_year,year,yield_bu_per_acre
0,11,1,17001,273.936458,0.187208,71.783333,7.191125,3.372792,-2.079917,2.556042,444.233333,0.222625,4.736086e-07,0.310632,0.317329,0.258307,2018,2018,197.8
1,11,1,17003,278.042667,18.158111,90.355556,4.224000,2.226778,-1.714222,0.091889,61.733333,0.099333,3.250581e-04,0.383857,0.324059,0.241359,2018,2018,188.4
2,11,1,17007,273.233000,6.926667,78.400000,5.291667,2.995000,-1.246667,2.581333,358.500000,0.154000,3.930181e-07,0.339598,0.347218,0.311294,2018,2018,194.1
3,11,1,17009,274.035000,0.126250,73.925000,6.281875,3.229250,-2.013000,2.428250,375.025000,0.204250,8.822729e-07,0.299224,0.304276,0.238118,2018,2018,187.9
4,11,1,17011,273.145593,4.493370,76.974074,5.554111,3.273741,-1.988667,2.418407,234.255556,0.165778,5.040850e-07,0.338929,0.348143,0.320493,2018,2018,214.1


## Обработка `NaN` значений

In [7]:
data[data.isna().any(axis=1)]

,month,day,fips,temperature_min,precipitation,humidity_relative,wind_gust,wind_speed,wind_u_component,wind_v_component,solar_radiation_downward,vapor_pressure_deficit,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,target_year,year,yield_bu_per_acre
17219,1,1,17043,270.239500,7.914000,92.600000,10.668167,5.038333,-1.216167,-4.847500,36.550000,0.048000,NaN,NaN,NaN,NaN,2022,2022,184.7
17410,1,15,17043,266.716667,1.298500,82.633333,9.850833,5.719000,-4.128000,-3.938667,1762.183333,0.075667,NaN,NaN,NaN,NaN,2022,2022,184.7
17601,2,1,17043,271.491500,4.346333,70.500000,11.133000,4.246667,-0.064333,3.682833,1647.450000,0.210167,NaN,NaN,NaN,NaN,2022,2022,184.7
17792,2,15,17043,279.375000,0.004333,64.000000,0.000000,3.984000,1.909000,3.496000,0.000000,0.342000,NaN,NaN,NaN,NaN,2022,2022,184.7
17983,3,1,17043,276.278333,0.000000,65.916667,5.327667,2.171000,0.941833,0.530167,1811.216667,0.337333,NaN,NaN,NaN,NaN,2022,2022,184.7
18174,3,15,17043,274.858667,0.000000,72.766667,4.052500,1.726167,-0.952167,-0.813833,4740.033333,0.330333,NaN,NaN,NaN,NaN,2022,2022,184.7
18365,4,1,17043,271.952833,1.117000,77.183333,6.752833,3.056833,2.441667,-1.619833,3681.150000,0.177000,NaN,NaN,NaN,NaN,2022,2022,184.7
18556,4,15,17043,277.195667,0.400000,44.516667,10.693167,4.495167,3.973833,1.918500,3418.650000,0.553833,NaN,NaN,NaN,NaN,2022,2022,184.7
18747,5,1,17043,282.533000,9.022500,79.350000,10.789000,4.672667,2.943500,3.176167,839.616667,0.291500,NaN,NaN,NaN,NaN,2022,2022,184.7
18938,5,15,17043,291.173833,0.319167,56.600000,5.326667,2.180333,0.508500,-0.195500,3551.866667,1.134833,NaN,NaN,NaN,NaN,2022,2022,184.7


Не хватает данных для одного округа с `fips` 7043 о влажности почвы. Можем их удалить, т.к. это всего ли -1 объект (объектом здесь понимается временной ряд длиною в год для округа)

In [8]:
data.dropna(inplace=True)

## Разделяем на обучающую и тестовую выборку

In [9]:
data.drop("target_year", axis=1, inplace=True)

In [10]:
mask = data["year"] == data["year"].max()
data_train = data[~mask]
data_test = data[mask]

X_train = data_train.drop("yield_bu_per_acre", axis=1)
y_train = data_train["yield_bu_per_acre"]
X_test = data_test.drop("yield_bu_per_acre", axis=1)
y_test = data_test["yield_bu_per_acre"]

In [11]:
print(f"X_train: {X_train.shape[0]}")
print(f"y_train: {y_train.shape[0]}")
print(f"X_test: {X_test.shape[0]}")
print(f"y_test: {y_test.shape[0]}")
print(f"(test size)/(data size): {X_test.shape[0] / data.shape[0]}")

X_train: 16564
y_train: 16564
X_test: 4436
y_test: 4436
(test size)/(data size): 0.21123809523809522


In [12]:
columns_order = ["year", "month", "day", "fips"] + np.sort(
    X_train.columns.drop(["year", "month", "day", "fips"])
).tolist()
X_train = X_train[columns_order]
X_test = X_test[columns_order]

In [13]:
X_train.head(1)

,year,month,day,fips,humidity_relative,precipitation,skin_reservoir_content,soil_water_vol_layer1,soil_water_vol_layer2,soil_water_vol_layer3,solar_radiation_downward,temperature_min,vapor_pressure_deficit,wind_gust,wind_speed,wind_u_component,wind_v_component
0,2018,11,1,17001,71.783333,0.187208,4.736086e-07,0.310632,0.317329,0.258307,444.233333,273.936458,0.222625,7.191125,3.372792,-2.079917,2.556042


## Сохраняем

In [14]:
if not PATH_PROCESSED.exists():
    PATH_PROCESSED.mkdir()
X_train.to_csv(PATH_PROCESSED / "X_train.csv", index=False)
y_train.to_csv(PATH_PROCESSED / "y_train.csv", index=False)
X_test.to_csv(PATH_PROCESSED / "X_test.csv", index=False)
y_test.to_csv(PATH_PROCESSED / "y_test.csv", index=False)